# Web scrapper code

The following code was used for extracting data from [Vgchartz website](http://www.vgchartz.com/gamedb/). This site has the following information about video games:

* **Basic information:** name of the videogame, console, publisher and developer.
* **Scores:** by critics, users and vgchartz. This project only used information about the critics' and users' scores.
* **Sales:** total sales, total sales in North America, total sales in PAL region (see note above), total sales in Japan, other sales. All sales are in million dollars.
* **Release date:** this includes day, month and year displayed in the following format: 10th Nov 19
* All information can be searched by **'genre'**. 

_Note. PAL region refers to regions in Latin America, Middle East and Africa._

The code first loops through a list of genres and then through all pages of each genre. To reduce the number of requests, the total number of results was set to 200. Beautiful soup was used to parse the HTML content.

Important considerations about the code:
* To avoid overloading the server, randint() function from Python's random module was used. This function generates integers within a specified interval. Also sleep() function was used to pause the execution of the loop.
* The frequency of requests and the status code of the requests are used as a way to monitor the scrapping process and to make sure that the program is not overlading the server. In case the status code of the request is not '200' the loop breaks.

In [4]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from random import randint
from time import time
from IPython.core.display import clear_output

def scrap(genrelist, genrelen):
    #create lists that represent the columns of the database
    genre = []
    #num = []
    link = []
    name = []
    console = []
    publisher = []
    developer = []
    critic_sc = []
    user_sc = []
    t_sales = []
    na_sales = []
    pal_sales = []
    jap_sales = []
    oth_sales = []
    release = []
    update = []
        
    #set a starting time; this will be used to monitor the scrapping process
    start_time = time()
    req = 0
               
    #loop genres
    for genres in genrelist:
        #loop pages 
        for page in range(1, int(genrelen[genres])+1):
            url = "http://www.vgchartz.com/games/games.php?page="+str(page)+"&results=200&name=&console=&keyword=&publisher=&genre="+genres+"&order=CriticScore&ownership=Both&boxart=Both&banner=Both&showdeleted=&region=All&goty_year=&developer=&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=1&showreleasedate=1&showlastupdate=1&showvgchartzscore=0&showcriticscore=1&showuserscore=1&showshipped=0&alphasort=&showmultiplat=No"
            r = requests.get(url)
            #pause the loop to avoid overloading the server
            sleep(randint(5,8)) 

            #monitor the scraping process 
            req +=1
            elapsed_time = time() - start_time
            print('Request: {} | Frequency: {} requests/s'.format(req, req/elapsed_time))
            #clear output after each iteration and replace it with information about the most recent request
            clear_output(wait = True)
            # warn in case I get a non-200 response
            if r.status_code != 200:
                print('Request: {} | Status code: {}'.format(req, r.status_code))
                break
            #end of monitor        
            
            soup = BeautifulSoup(r.content, 'lxml')
            info = soup.find('div', id='generalBody').find('table')
            for row in info.find_all('tr')[3:]:
                item = row.find_all('td')
                #extract all information 
                x1 = item[0].text #num
                x2 = item[2].find('a')['href'].strip() #link of video game
                x3 = item[2].find('a').text.strip() #name of video game
                x4 = item[3].find('img')['alt'].strip() #console
                x5 = item[4].text.strip() #publisher
                x6 = item[5].text.strip() #developer
                x7 = item[6].text.strip() #critic score
                x8 = item[7].text.strip() #user score
                x9 = item[8].text.strip() #total sales
                x10 = item[9].text.strip() #sales in north america
                x11 = item[10].text.strip() #sales in PAL region
                x12 = item[11].text.strip() #sales in japan
                x13 = item[12].text.strip() #other sales
                x14 = item[13].text.strip() #release date
                x15 = item[14].text.strip() #last update
                #append information to lists
                #num.append(x1)
                link.append(x2)
                name.append(x3)
                console.append(x4)
                publisher.append(x5)
                developer.append(x6)
                critic_sc.append(x7)
                user_sc.append(x8)
                t_sales.append(x9)
                na_sales.append(x10)
                pal_sales.append(x11)
                jap_sales.append(x12)
                oth_sales.append(x13)
                release.append(x14)
                update.append(x15)
                genre.append(genres)
        
    #Create a dictionary with all columns. Include a column of genre.
    data_dict = {'Genre':genre,'Name': name,'Console':console,'Publisher':publisher,'Developer': developer,
                 'Critic_score':critic_sc,'User_score':user_sc,'Total_sales':t_sales,'America_sales':na_sales,
                 'PAL_sales':pal_sales, 'Japan_sales':jap_sales,'Other_sales':oth_sales,'Release_date':release,
                 'Update_date':update}
        
    df = pd.DataFrame.from_dict(data_dict)
    return df 

## Sample for testing the scrapper

Our total of requests will be 305. Therefore, it is better to try the code before extracting all information.  

In [5]:
#A small sample is created with just 2 genres and 2 requests per genre
small_genrelist = ["Action", "Action-Adventure"]
small_genrelen = {'Action':'2','Action-Adventure':'2'}

In [6]:
#Use this to extract data for the small sample
df0 = scrap(small_genrelist, small_genrelen)

Request: 4 | Frequency: 0.11595316887560933 requests/s


In [7]:
#Use this to display the first 10 rows of the data frame
df0[:10]

,Genre,Name,Console,Publisher,Developer,Critic_score,User_score,Total_sales,America_sales,PAL_sales,Japan_sales,Other_sales,Release_date,Update_date
0,Action,Final Fight,SNES,Capcom,Capcom,10.0,N/A,1.56m,0.67m,0.17m,0.69m,0.03m,10th Nov 91,N/A
1,Action,Red Dead Redemption: Undead Nightmare,PS3,Rockstar Games,Rockstar San Diego,10.0,N/A,1.18m,0.47m,0.45m,0.06m,0.19m,23rd Nov 10,N/A
2,Action,Red Dead Redemption: Undead Nightmare,X360,Rockstar Games,Rockstar San Diego,10.0,N/A,1.11m,0.61m,0.38m,0.02m,0.10m,23rd Nov 10,N/A
3,Action,Grand Theft Auto IV,PS3,Rockstar Games,Rockstar North,10.0,9.0,10.57m,4.79m,3.73m,0.44m,1.62m,29th Apr 08,N/A
4,Action,Grand Theft Auto IV,X360,Rockstar Games,Rockstar North,10.0,N/A,11.09m,6.80m,3.11m,0.14m,1.04m,29th Apr 08,N/A
5,Action,Rockstar Games Double Pack: Grand Theft Auto I...,XB,Rockstar Games,Rockstar North,9.8,N/A,2.49m,1.84m,0.56m,N/A,0.09m,31st Oct 03,N/A
6,Action,Grand Theft Auto V,PS4,Rockstar Games,Rockstar North,9.7,N/A,19.39m,6.06m,9.71m,0.60m,3.02m,18th Nov 14,03rd Jan 18
7,Action,God of War (2018),PS4,Sony Interactive Entertainment,SIE Santa Monica Studio,9.7,10.0,6.15m,2.83m,2.17m,0.13m,1.02m,20th Apr 18,03rd Dec 18
8,Action,Rockstar Games Double Pack: Grand Theft Auto I...,PS2,Rockstar Games,Rockstar North,9.7,N/A,1.72m,0.85m,0.66m,N/A,0.22m,21st Oct 03,N/A
9,Action,Resident Evil 4,PS2,Capcom,Capcom Production Studio 4,9.6,9.3,3.62m,2.08m,0.83m,0.46m,0.25m,25th Oct 05,N/A


In [8]:
#Use this to store the data in a csv
df0.to_csv("games_sample.csv")

## Complete database

The code works, therefore, we can extract all information with the 305 requests.
**Caution:** The code will take a while to extract all information. It could last up to 30 to 40 minutes.

In [9]:
genres = ["Action", "Action-Adventure", "Adventure", "Board+Game", "Education", "Fighting", 
         "Misc", "MMO", "Music","Party","Platform", "Puzzle", "Racing", "Role-Playing", "Sandbox", 
         "Shooter", "Simulation","Sports", "Strategy", "Visual+Novel"]

genres_len = {'Action':'39','Action-Adventure':'4','Adventure':'28','Board+Game':'1','Education':'1','Fighting':'11',
             'Misc':'47','MMO':'1','Music':'2','Party':'1','Platform':'18','Puzzle':'17','Racing':'16','Role-Playing':'24',
             'Sandbox':'1','Shooter':'24','Simulation':'14','Sports':'27','Strategy':'27','Visual+Novel':'2'}

In [10]:
dfall = scrap(genres, genres_len)

Request: 305 | Frequency: 0.12416177727787629 requests/s


In [11]:
#Store the extracted data into a csv 
dfall.to_csv("vgames.csv")

In [12]:
#check the length
len(dfall)

56897

In [13]:
#check the type of the data
type(dfall)

pandas.core.frame.DataFrame

In [14]:
#Check number of rows and columns
dfall.shape

(56897, 14)